# Exercise 2
Today we are going to continue to work on point clouds.
We will work on clustering point clouds. That enables us to segment them.

In [1]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans, k_means

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def draw_labels_on_model(pcl,labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    print("%s has %d clusters" % (pcl_name, max_label + 1))
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])



## K-means on a cube
We created a point cloud using `open3d`.
Our goal is to segment each side using k-means.

In [3]:
pcl_name = 'Cube'
density = 1e4 # density of sample points to create
pcl = o3d.geometry.TriangleMesh.create_box().sample_points_uniformly(int(density))
eps = 0.4
print("%s has %d points" % (pcl_name, np.asarray(pcl.points).shape[0]))
o3d.visualization.draw_geometries([pcl])

Cube has 10000 points


If we just use k-means out of the box with the point cloud, we will get what just has been visualized.

Note: Using the '+' and '-' keys in the viewer will increase/decrease the size of the points.

In [4]:
km = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
xyz = np.asarray(pcl.points)
labels = km.fit_predict(xyz)
draw_labels_on_model(pcl, labels)
print(labels)

Cube has 6 clusters
[5 5 4 ... 4 5 4]


We can see that we get six clusters, but they do not span a side.

We try again, but this time we instead use the normals of the cube as input for k-means.

The normals for each plane should be parallel with the other normals from said plane.

In [5]:
###
# Code goes here
###
pcl.estimate_normals()
normals = np.asarray(pcl.normals)
xyz_normals = np.hstack((xyz, normals))
labels = km.fit_predict(xyz_normals)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


This still does not work, opposite sides will also have normals that point the other way ($\vec{n}$ and $-\vec{n}$).

So, to combat this we can attempt to use the xyz coordinates and the normals.

## More exercises

### A) K-means continued.

Combine the point cloud points (xyz) with the normals and do k-means.

```xyz_n = np.concatenate((xyz, normals), axis=1)```

Do you get better clusters?
Why would adding the normals help?

### B) 
Try weighting either the points or normals by scaling them by some factor. Can this perfectly segment each of the faces of the cube?
### C)
Try to cluster all the different shapes using k means.
```{Python}
d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=1).translate(
    (0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_moebius(twists=2).translate(
    (d, -d, 0))
mesh.sample_points_uniformly(int(1e5)), 0.5
```

### D)
Now try segmenting a different point cloud located at `pointclouds/fragment.ply`
Are you able to cluster the point cloud?

Which features could be useful to segment this point cloud?
- fpfh features?
- xyz
- normals 
- colors

Are you able to get clusters that make sense? Why?

### E)
Use the built-in `cluster_dbscan` algorithm.
Tweak the parameters and see what you get out.

Attempt on the combined figures and on `fragment.ply`
```{Python}
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.02

#min_points (int) – Minimum number of points to form a cluster.
min_points = 10

labels = np.array(pcl.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
```


In [6]:
# A
xyz_n = np.concatenate((xyz, normals), axis=1)
labels = km.fit_predict(xyz_n)
draw_labels_on_model(pcl, labels)


Cube has 6 clusters


In [7]:
# B
scale_xyz = 4
scale_normals = 2
xyz_n = np.concatenate((xyz * scale_xyz, normals * scale_normals), axis=1)
labels = km.fit_predict(xyz_n)
draw_labels_on_model(pcl, labels)

Cube has 6 clusters


In [8]:
# C
d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=1).translate(
    (0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=2).translate(
    (d, -d, 0))
mesh = mesh.sample_points_uniformly(int(1e5), 0.5)

o3d.visualization.draw_geometries([mesh])



In [11]:
km2 = KMeans(n_clusters=6, init='random',
            n_init=50, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray
mesh_xyz = np.asarray(mesh.points)
mesh_labels = km2.fit_predict(mesh_xyz)
draw_labels_on_model(mesh, mesh_labels)



Cube has 6 clusters


In [13]:
# D
pc = o3d.io.read_point_cloud("TestData/fragment.ply")
xyz = np.asarray(pc.points)
# o3d.visualization.draw_geometries([pc])

km3 = KMeans(n_clusters=6, init='random',
            n_init=10, max_iter=300, tol=1e-04, random_state=0)

# Get the points from the pointcloud as nparray

## -- TRY WITH POINTS -- ##
labels = km3.fit_predict(xyz)
draw_labels_on_model(pc, labels)

## -- TRY WITH POINTS AND NORMALS -- ##
pc.estimate_normals()
normals = np.asarray(pc.normals)
xyz_normals = np.hstack((xyz, normals))
scale_xyz = 4
scale_normals = 2
xyz_n = np.concatenate((xyz * scale_xyz, normals * scale_normals), axis=1)
labels = km3.fit_predict(xyz_n)
draw_labels_on_model(pc, labels)

## -- TRY WITH POINTS AND RGB -- ##
xyz_rgb = np.hstack((xyz, pc.colors))
labels = km3.fit_predict(xyz_rgb)
draw_labels_on_model(pc, labels)

## -- TRY WITH ALL COMBINED -- ##
all = np.hstack((xyz, normals, pc.colors))
labels = km3.fit_predict(all)
draw_labels_on_model(pc, labels)

Cube has 6 clusters
Cube has 6 clusters
Cube has 6 clusters
Cube has 6 clusters


In [19]:
## -- TRY WITH ALL COMBINED -- ##
km3 = KMeans(n_clusters=12, init='random',
            n_init=25, max_iter=300, tol=1e-04, random_state=0)
all = np.concatenate((xyz, normals, pc.colors), axis=1)
labels = km3.fit_predict(all)
draw_labels_on_model(pc, labels)

Cube has 12 clusters


This needs further investigatin an a way better idea...

In [21]:
# E 
pcl = o3d.io.read_point_cloud("TestData/fragment.ply")
#eps (float) – Density parameter that is used to find neighbouring points.
eps = 0.02

#min_points (int) – Minimum number of points to form a cluster.
min_points = 10

labels = np.array(pcl.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

draw_labels_on_model(pcl, labels)

Precompute neighbors.[================>           Cube has 10 clusters
Precompute neighbors.[========================================] 100%


In [23]:
eps = 0.2

min_points = 10
labels = np.array(mesh.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

draw_labels_on_model(mesh, labels)

Cube has 6 clustersPrecompute neighbors.[=>                                      ] 2%
Precompute neighbors.[========================================] 100%
Clustering[========================================] 100%
